In [ ]:
import datetime, time
import numpy as np
import pandas as pd
import datetime as dt
import os
import json

from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import *
from bokeh.plotting import figure, output_file, curdoc, show as bshow

from bokeh.models import DatetimeTickFormatter, Legend, NumeralTickFormatter
from bokeh.layouts import widgetbox, row, column, layout, gridplot
from bokeh.models import ColumnDataSource, Toggle, BoxAnnotation, CustomJS
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Div

from bokeh.io import curdoc, output_notebook
from bokeh.plotting import show, figure

# koolpex related variables
api_url = os.path.join("https://", os.getenv("SERVERNAME"), os.getenv("REPORT_URL"))

full_url = os.path.join(api_url, "bps")

When embedding it:

This goes to the `<head>`:
```
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-2.0.1.min.js"></script>
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-widgets-2.0.1.min.js"></script>
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-tables-2.0.1.min.js"></script>
```

And this goes to the `<body>`:
```
    <body>
      <div id="myplot"> </div>
      <script>
         fetch({{full_url}})
           .then(function(response) { return response.json(); })
           .then(function(item) { Bokeh.embed.embed_item(item); })
      </script>

    </body>
```

In [ ]:
# output_notebook()

In [ ]:
fname = 'data/buildbps.csv'
df = pd.read_csv(fname)#, parse_dates=[1])

# Cleaning and reformatting
for s in [' BPS_Water_Bill ', ' BPS_Gas_Bill ', ' BPS_Electric_Bill ']:
    df[s[1:-1]] = df[s].replace( '\$-','0', regex=True ).replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)
    
    wrong_lon = np.where(df['SMMA_longitude']>0)[0][0]
df['SMMA_longitude'].loc[wrong_lon] = df['SMMA_longitude'].loc[wrong_lon]*-1
# list(df.columns)

In [ ]:
keep_columns = ['BPS_School_Name', 'SMMA_latitude', 'SMMA_longitude',
                'SMMA_FA_Windows', 'SMMA_FA_Boilers', 'SMMA_FA_Heating_Distribution_Systems',
                'SMMA_FA_Ventilation_Distribution_Systems',
                'BPS_Water_Bill', 'BPS_Gas_Bill', 'BPS_Electric_Bill']
                
df = df[keep_columns]       
df.head().T

In [ ]:
# for debugging
REQUEST = json.dumps({
 'args' : {}
})

In [ ]:
# GET /help

resp = {'Help': """When embedding it:

This goes to the `<head>`:
```
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-1.4.0.min.js"></script>
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-widgets-1.4.0.min.js"></script>
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-tables-1.4.0.min.js"></script>
```

And this goes to the `<body>`:
```
    <body>
      <div id="myplot"> </div>
      <script>
         fetch({{full_url}})
           .then(function(response) { return response.json(); })
           .then(function(item) { Bokeh.embed.embed_item(item); })
      </script>

    </body>
```"""}

print(json.dumps(resp))

In [ ]:
# # ResponseInfo GET /help
# print(json.dumps({
#     "headers" : {
#         "Content-Type" : "application/json"
#     },
#     "status" : 200
# }))


In [ ]:
# GET /bps
req = json.loads(REQUEST)
args = req['args']

data_source = ColumnDataSource(df)
TOOLS = "box_select,lasso_select, reset"

topleft = figure(tools=TOOLS, plot_width=400, plot_height=400, title="Geographic location")
topleft.scatter("SMMA_longitude", "SMMA_latitude", source=data_source, color='red', size=10)

topright = figure(tools=TOOLS, plot_width=400, plot_height=400,
                y_axis_type="log", x_axis_type="log", title="Water-Gas correlation")
topright.circle("BPS_Water_Bill", "BPS_Gas_Bill", source=data_source, size=10, color="navy",
                alpha=0.5, nonselection_fill_alpha=0.05)
topright.yaxis.formatter = NumeralTickFormatter(format="0,0")
topright.xaxis.formatter = NumeralTickFormatter(format="0,0")

bottomright = figure(tools=TOOLS, plot_width=400, plot_height=400, 
                     y_axis_type="log", x_axis_type="log", title="Water-Electric correlation")
bottomright.square("BPS_Water_Bill", "BPS_Electric_Bill", source=data_source,
                   size=10, color="olive", alpha=0.5, nonselection_fill_alpha=0.05)
bottomright.yaxis.formatter = NumeralTickFormatter(format="0,0")
bottomright.xaxis.formatter = NumeralTickFormatter(format="0,0")

bottomleft = figure(tools=TOOLS, plot_width=400, plot_height=400, 
                    y_axis_type="log", x_axis_type="log", title="Electric-Gas correlation")
bottomleft.diamond("BPS_Electric_Bill", "BPS_Gas_Bill", source=data_source, size=15, color="green",
                alpha=0.5, nonselection_fill_alpha=0.05)
bottomleft.yaxis.formatter = NumeralTickFormatter(format="0,0")
bottomleft.xaxis.formatter = NumeralTickFormatter(format="0,0")

columns = [ TableColumn(field=tc, title=tc[4:]) 
           for tc in ["BPS_School_Name", "BPS_Water_Bill", "BPS_Electric_Bill", "BPS_Gas_Bill"]]
data_table = DataTable(
            source=data_source,
            columns=columns,
            width=800,
            height=800,
            editable=False,
            reorderable=True,
            #index_position=-1,
        )

Title = Div(text="<br><h2> Expenses of School Buildings in Boston</h2>")
p = column(Title,
           row(gridplot([[topleft, topright], [bottomleft, bottomright]]), data_table))

# show(p)

item_text = json.dumps(json_item(p, "myplot"))
print(item_text)

In [ ]:
# ResponseInfo GET /bps
print(json.dumps({
    "headers" : {
        "Access-Control-Allow-Origin" : "https://%s"%os.getenv("SERVERNAME"),
        "Origin" : "https://%s"%os.getenv("SERVERNAME"),
        "Access-Control-Expose-Headers": "Content-Length,API-Key"
    },
    "status" : 200
}))


All requests in bash command line with curl:
```
API_URL={{api_url}}
curl -s "$API_URL/bps"
```